In [1]:
import sys
import time
from pathlib import Path

import h5py as h5
import numpy as np
from scipy.ndimage import uniform_filter1d

sys.path.append('../lib')
from storage import get_storage_functions
from local_paths import preproc_dir

In [2]:
#============================================================================
# session
#============================================================================
sess_name = 'sess_name'

#============================================================================
# main
#============================================================================
sdf_method = 'mwa'  # moving window average
sdf_window = 50     # ms

#============================================================================
# paths
#============================================================================
proc_dir   = preproc_dir
output_dir = preproc_dir

In [3]:
# Parameters
sess_name = "Pa210201"
sdf_window = 1


# Check parameters and whether already done

In [4]:
rasters_path = Path(proc_dir) / (sess_name + '-rasters.nwb')
print('Loading rasters from', rasters_path)
rasters_path = rasters_path.expanduser()
assert rasters_path.is_file()

proc_path = Path(proc_dir) / (sess_name + '-proc.h5')
print('Loading shared processing from', proc_path)
proc_path = proc_path.expanduser()
assert proc_path.is_file()

assert sdf_method == 'mwa'  # only using mwa, but can implement others like gaussian filtering

output_dir = Path(output_dir)
assert output_dir.expanduser().is_dir()
output_path = output_dir / (sess_name + f'-sdf-{sdf_method}_{sdf_window}.h5')
print('Saving results to', output_path)
output_path = output_path.expanduser()

Loading rasters from ../test_data/Pa210201-rasters.nwb
Loading shared processing from ../test_data/Pa210201-proc.h5
Saving results to ../test_data/Pa210201-sdf-mwa_1.h5


In [5]:
if output_path.is_file():
    with h5.File(output_path, 'r') as f:
        try:
            if f['progress_report/sdf/all_done'][()].item():
                raise RuntimeError(f'{sess_name} has already been processed')
        except KeyError:
            pass

In [6]:
save_results, add_attr_to_dset, check_equals_saved, link_dsets, copy_group = \
    get_storage_functions(output_path)

In [7]:
save_results('config/sdf/method', sdf_method)
save_results('config/sdf/window', sdf_window, attrs=dict(unit='ms'))

# Main

In [8]:
with h5.File(rasters_path, 'r') as f:
    all_unit_names = f['processing/ecephys/unit_names/unit_name'][()].astype(str)
with h5.File(proc_path, 'r') as f:
    unit_names = f['unit_selection/simple'][()].astype(str)
unexpected_units = set(unit_names) - set(all_unit_names)
assert not len(unexpected_units), f'got {len(unexpected_units)} units in shared processing file absent from rasters data'
unit_sel = np.array([v in unit_names for v in all_unit_names])
unit_sel = np.nonzero(unit_sel)[0]

with h5.File(rasters_path, 'r') as f:
    rasters = f['processing/ecephys/rasters/data'][()]
rasters = rasters[:,unit_sel]

In [9]:
sdf = rasters.astype(np.float32) * 1e3
if sdf_window == 1:
    pass
else:
    print(f'calculating sdf with a {sdf_window} ms moving average window...', end=' ')
    t0 = time.time()
    sdf = uniform_filter1d(sdf, sdf_window, axis=0)
    print(f'done ({time.time()-t0:.1f} s)')
    print()

print('spike density function:', sdf.shape, sdf.dtype)

spike density function: (6234150, 92) float32


In [10]:
save_results('sdf', sdf, attrs=dict(
    unit_names=unit_names.astype(bytes),
    unit='spikes/s',
    smoothing_method=sdf_method,
    smoothing_window=sdf_window))

# Wrap up

In [11]:
save_results(f'progress_report/sdf/all_done', True)

In [12]:
%load_ext watermark
%watermark -vm --iversions -rbg

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.12.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-92-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit

Git hash: b0bb10f45dee065cc3af96fe224326b883d27431

Git repo: https://github.com/willwx/free_viewing_staging.git

Git branch: master

sys  : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
numpy: 1.24.3
h5py : 3.8.0

